In [167]:
import vk
from html2text import html2text
import re
import json
import os

In [168]:
VK_SECRET = 'VeKnPzXVGajCv******'
VK_KEY = '381fa545381fa545381fa5458f387eab6e3381f381fa5456267ce2d46086178c8******'

GROUPS_FOLDER = 'groups'

In [127]:
vkname_subs_re = re.compile('\[id\d+\|[^\]]+\]')



'@@NAME, вы есть будите, то что я приготовлю!) А не на троих соображать😜😂'

In [35]:
session = vk.Session(access_token=VK_KEY)
api = vk.API(session)


[{'first_name': 'Павел', 'last_name': 'Дуров', 'uid': 1}]

In [142]:
def text_preprocess(raw_text):
    text = vkname_subs_re.sub('@@NAME', raw_text)
    text = html2text(text)
    return text

In [132]:
group_id = 113651367



In [152]:
def download_messages_from_group(group_id):
    messages = []
    all_msg_count, *items = api.wall.get(owner_id=-group_id, count=1)
    print('all_msg_count', all_msg_count)
    msg_processed_count = 0
    
    while all_msg_count > msg_processed_count:
        _, *items = api.wall.get(owner_id=-group_id, count=100, offset=msg_processed_count)
        print(len(items))
        msg_processed_count += len(items)        

        for i in items:
            assert i['post_type'] == 'post', i
            if 'is_pinned' in i:
                continue

            messages.append(text_preprocess(i['text']))    

            replay_count, *replays = api.wall.getComments(owner_id=-group_id, post_id=i['id'], count=100)
            for r in replays:
                messages.append(text_preprocess(r['text']))


    return messages

In [154]:
msgs = download_messages_from_group(160967118)

all_msg_count 3
3


In [173]:
group_link = 'https://vk.com/event113651367'

In [174]:

group_name = re.findall('event\d+', group_link)[0]
group_id = int(re.findall('\d+', group_name)[0])
group_file_name = group_name + '.jsonl'

group_all_messages = download_messages_from_group(group_id)
with open(os.path.join(GROUPS_FOLDER, group_file_name), 'w') as f:
    for m in group_all_messages:
        print(json.dumps({'text': m}), file=f)
    print('Wrote {} messages to {}'.format(len(group_all_messages), group_file_name))

all_msg_count 188
100
88


AssertionError: 

In [178]:
2000 * 3 * 10 / 8 * 4

30000.0